# Machine Learning - Assessment 2

Through the following notebook, you will be tasked with **implementing** two different classification approaches: **k Nearest Neighbours** and **Decision Trees**. You will apply these to two different datasets, evaluate the produced models and analyse their performance.

This will be done through several different tasks and sub-tasks:
- [Dataset](#Dataset)
    - [Task 1: Dataset statistics **(10\%)**](#Task-1---Dataset-statistics)
- [k Nearest Neigbours](#k-Nearest-Neigbours)
    - [Task 2.1: Euclidean distance **(10\%)**](#Task-2.1---Euclidean-distance)
    - [Task 2.2: kNN classifier **(20\%)**](#Task-2.2---kNN-classifier)
- [Decision Trees](#Decision-Trees)
    - [Task 3.1: Entropy and Information Gain **(10\%)**](#Task-3.1---Entropy-and-Information-Gain)
    - [Task 3.2: Decision Tree classifier **(30\%)**](#Task-3.2---Decision-Tree-classifier)
- [Model evaluation and analysis](#Model-evaluation-and-analysis)
    - [Task 4.1: Evaluation **(10\%)**](#Task-4.1---Evaluation)
    - [Task 4.2: Analysis **(10\%)**](#Task-4.2---Analysis)

**Note:** The (%) noted above are out of 100; this will be scaled down to **maximum of 70 marks** for the assessment.

In [ ]:
import math
import numpy as np
import pandas as pd

## Dataset

For this assessment, you will be working with two datasets:
- A [numeric-only](#Numerical-data) dataset (crystal systems of Li-ion batteries)
- A dataset with [mixed numeric and categorical](#Categorical-data) features (forest cover)

For developing your solutions, **you have only been provided with a portion of the samples from each of the dataset** (70% of the [numeric-only](#Numerical-data) and 50% of the [mixed numeric and categorical](#Categorical-data) data). These will be in **identical format** to the full datasets used for evaluating your solutions (there will just be more samples).

### Numerical data

The main part of the assessment will be done on the dataset containing only numerical features describing the physical and chemical properties of the Li-ion battery, which can be classified on the basis of their crystal system. Three major classes of crystal systems are: _monoclinic_, _orthorhombic_ and _triclinic_. (The dataset for this assessment has been adapted from the full dataset which can be found [here](https://www.kaggle.com/datasets/divyansh22/crystal-system-properties-for-liion-batteries

Each sample corresponds to the properties of a battery, and consists of following features:

| Feature Name      | Value | Description |
| :---------------- | :----- | ----------- |
| `Formation Energy`       | `float`: eV | Formation energy of the material. |
| `E Above Hull` | `float`: eV | Energy of decomposition of material into most stable ones. |
| `Band Gap` | `float`: eV | Band gap. |
| `Nsites` | `int`: count | Number of atoms in the unit cell of the crystal. |
| `Density` | `float`: gm/cc | The density of bulk crystalline materials. |
| `Volume` | `float` | The unit cell volume of the material. |

The goal for the assessment is to predict whether the crystal system of the battery is _monoclinic_, _orthorhombic_ or _triclinic_, which provides a classification for each sample:

| Class      | Value | Description |
| :---------------- | :----- | ----------- |
| `Crystal System`  | `string`: class designation | Class of the crystal system (three different values). |


This dataset will be used for the developmet and testing of both the [kNN](#k-Nearest-Neigbours) and [Decision Tree](#Decision-Trees) classifier, as well as for the [evaluation](#Task-4.1---Evaluation) of both of the classification approaches.

The following cell loads the dataset, and splits the samples randomly, using  $80\%$ of samples for training and $20\%$ for testing:

In [ ]:
from sklearn.model_selection import train_test_split
from itertools import count

numerical_data = pd.read_csv(r'batteries.csv')

print("The dataset consists of {} samples".format(len(numerical_data)))
print("The numerical data loaded consists of following columns:\n{}".format(
    "\n".join(["\tColumn {}: {}".format(i, x) for i, x in enumerate(numerical_data.columns)])))
print("An example sample:\n{}".format(
    "\n".join(["\tFeature {}: {}={}".format(i, x, v) for i, (x, v) in 
               enumerate(zip(numerical_data.columns, numerical_data.loc[0, :])) if not x == 'Crystal System'])))
print('\tClass (Family): {}'.format(numerical_data.loc[0, 'Crystal System']))

# We split the dataset into features and the target class, using the 'Family' column as the target:
X_nd = numerical_data.loc[:, numerical_data.columns!='Crystal System'].values
y_nd = numerical_data['Crystal System'].values

# We further split the dataset into a training and testing set
X_nd_train, X_nd_test, y_nd_train, y_nd_test = train_test_split(X_nd, y_nd, test_size = 0.2, random_state = 0)

print("The training set consists of {} samples and {} associated ground truth classes".format(len(X_nd_train),
                                                                                             len(y_nd_train)))
print("The test set consists of {} samples and {} associated ground truth classes".format(len(X_nd_test),
                                                                                             len(y_nd_test)))

### Categorical data

The second dataset for this assessment contains a mix of numerical and categorical features, with the goal of predicting the forest cover type. The data used in this assessment is adapted from a full dataset provided [here](https://archive.ics.uci.edu/ml/datasets/Covertype) (Jock A. Blackard and Colorado State University).

Each sample corresponds to a 30x30 meter cell, and consists of cartographic variables only (i.e. it does not rely on any remotely sensed imaging data), derived from from US Geological Survey (USGS) and USFS data. Each sample consists of the following features (categorical features can be distinguished by being encoded as a string rather tha a numer):

| Feature Name      | Value | Description |
| :---------------- | :----- | ----------- |
| `Elevation`       | `float`: kilometers | Elevation |
| `Aspect`   | `int`: degrees azimut | Aspect in degreez azimuth |
| `Slope`      | `int`: degrees | Slope in degrees |
| `Horizontal_Distance_To_Hydrology`   | `float`: kilometers | Horiz. distance to nearest surface water feature        |
| `Vertical_Distance_To_Hydrology`   | `float`: kilometers | Vert. distance to nearest surface water feature        |
| `Horizontal_Distance_To_Roadways`     | `float`: kilometers | Hor. distance to nearest roadway       |
| `Hillshade_9am`   |`int`: $0 \dots 255 $|  Hillshade index at 9am, summer solstice        |
| `Hillshade_Noon`      |`int`: $0 \dots 255 $| Hillshade index at noon, summer solstice       |
| `Hillshade_3pm`   |`int`: $0 \dots 255 $| Hillshade index at 3pm, summer solstice        |
| `Horizontal_Distance_To_Fire_Points`  | `float`: kilometers | Horiz. distance to nearest wildfire ignition point        |
| `Wilderness_Area`   | `string`: name | Wilderness area name (4 different values)       |
| `Soil_Type`   |`string`: code| Soil type code (40 different values)       |

The goal of this dataset is to predict the forrest cover type for each 30x30 meter cell. A class, corresponding to the forest cover type, is associated to each sample:

| Class      | Value | Description |
| :---------------- | :----- | ----------- |
| `Cover_Type`       | `string`: name | Forest cover type designation (7 different values). |


**Note:** Tackling the categorical dataset is a **stretch task** for this assessment. It will be used in the development and testing of the [Decision Tree](#Decision-Trees) classifier, as well as for their [evaluation](#Task-4.1---Evaluation). Tackling this data will **require you to implement a** `DecisionTree` **from scratch**, as `sklearn` **does not provide this functionality**.

The following cell loads the dataset, and splits the samples randomly, using  $80\%$ of samples for training and $20\%$ for testing:

In [ ]:
categorical_data = pd.read_csv(r'forestcover.csv')

print("The dataset consists of {} samples".format(len(categorical_data)))
print("The numerical data loaded consists of following columns:\n{}".format(
    "\n".join(["\tColumn {}: {}".format(i, x) for i, x in enumerate(categorical_data.columns)])))
print("An example sample:\n{}".format(
    "\n".join(["\tFeature {}: {}={}".format(i, x, v) for i, (x, v) in 
               enumerate(zip(categorical_data.columns, categorical_data.loc[0, :])) if not x == 'Cover_Type'])))
print('\tClass (Cover_Type): {}'.format(categorical_data.loc[0, 'Cover_Type']))

# We split the dataset into features and the target class, using the 'Cover_Type' column as the target:
X_cd = categorical_data.loc[:, categorical_data.columns!='Cover_Type'].values
y_cd = categorical_data['Cover_Type'].values


# We further split the dataset into a training and testing set
X_cd_train, X_cd_test, y_cd_train, y_cd_test = train_test_split(X_cd, y_cd, test_size = 0.2, random_state = 0)

print("The training set consists of {} samples and {} associated ground truth classes".format(len(X_cd_train),
                                                                                             len(y_cd_train)))
print("The test set consists of {} samples and {} associated ground truth classes".format(len(X_cd_test),
                                                                                             len(y_cd_test)))

### Task 1 - Dataset statistics
#### 10% marks

To handle the data for the classification tasks, implement functions that will assist in querying some basic information about the dataset you are handling.

Given a set of M samples $\mathbf{x} = \{\mathbf{x}_i\} \forall i=1..M$ with n features each $\mathbf{x}_i = \{x_{1,i}, \dots, x_{n,i}\}$, and the corresponding ground truth labels $\mathbf{y} = \{y_i\} \forall i=1..M$, implement two functions:

- `class_probabilities(y)` : calcualtes the class probabilities from the list of ground truth labels

    *Inputs:*<br>
    `y`: The list of ground truth labels for a dataset<br>
    
    *Returns:*<br>
    A dictionary of class labels and their probabilities (i.e. the frequency of that class in the dataset)

    *Example* :
    ```python
    y = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
    print(class_probabilities(y))
    ```
    *Example output*:<br>
    `{0: 0.3, 1: 0.5, 2: 0.2}`
    
    *Note:* Python dictionaries are unordered, so running this example might result in a different order of keys in the output, for example: `{2: 0.2, 0: 0.3, 1: 0.5}`<br><br>
   
- `most_common_class` : finds the most common class from the list of ground truth class labels.

    *Inputs:*<br>
    `y`: the list of ground truth labels for a dataset<br>
    
    *Returns:*<br>
    the label of the most common class represented in the dataset.
    
    *Example*:
    ```python
    y = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
    print(most_common_class(y))
    ```
    *Example output*:<br>
    `1`

#### SOLUTION CELLS:

In [ ]:
def class_probabilities(y):
    ################################
    ####### YOUR CODE HERE #########
    ################################

    return {} #  you should probably replace this line
    

In [ ]:
def most_common_class(y):
    ################################
    ####### YOUR CODE HERE #########
    ################################

    return 0 # you should probably replace this line
    

#### TESTING CELLS:

In [ ]:
unq, cnt = np.unique(y_nd, return_counts=True)
print("All unique class labels and counts from numerical-only dataset: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities:\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd).items()])))
print("Most common class in the numerical-only dataset: {}".format(most_common_class(y_nd)))
print("\n")

unq, cnt = np.unique(y_nd_train, return_counts=True)
print("All unique class labels for the numerical-only training set: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities (num-only training set):\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd_train).items()])))
print("Most common class in the num-only training set: {}".format(most_common_class(y_nd_train)))
print("\n")

unq, cnt = np.unique(y_nd_test, return_counts=True)
print("All unique class labels for the numerical-only testing set: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities (num-only tests set):\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd_test).items()])))
print("Most common class in the num-only test set: {}".format(most_common_class(y_nd_test)))
print("\n")

y_test = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
print("All class labels for a toy set: {}".format(y_test))
print("All classses and their probabilities for the toy set:\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_test).items()])))
print("Most common class in the toy set: {}".format(most_common_class(y_test)))
print("\n")

print("All classes and their probabilities from the categorical dataset:\n{}".format(
    "\n".join(["\t{}: {:.3f}".format(c, p) for c, p in class_probabilities(y_cd).items()])))
print("Most common class in the categorical dataset: {}".format(most_common_class(y_cd)))

## k Nearest Neighbours

For the first part of the assessment, you are required to implement **k Nearest Neigbours**. A key concept for kNN (and many different machine learning algorithms) is that of **distance**. 

### Task 2.1 - Euclidean distance
#### 10% marks

One of the most commonly used distance metrics is the Euclidean distance. The Euclidean distance between two points in the Euclidean space (also known as $L_2$ norm) is defined as the length of the line segment between those two points. For example, if we have two points in 2D space (i.e. two samples consisting of two features), $\mathbf{x}_0 = \{x_{0,0}, x_{0,1}\}$ and $\mathbf{x}_1 = \{x_{1,0}, x_{1,1}\}$, we can calculate the distance as:

\begin{equation*}
d_{\texttt{Euclidean}} = \sqrt{(x_{0,0} - x_{1,0})^2 + (x_{0,1} - x_{1,1})^2}
\end{equation*}

In other words, we need to calculate the difference between the first features of the two samples, the difference between the second features of the two samples, then square each of the differences, sum those squares, and calculate the square root of the resulting sum.

In general, when we are dealing with points in nD space (corresponding to samples with n features) of the form $\mathbf{x}_i = \{x_{0,i}, x_{1,i}, \dots, x_{n,i}\}$, the procedure is very similar. To calculade the Euclidean disatnce, you need to calculate the difference between the corresponding features of two samples, square each of the differences, and calculate the square root of the resulting sum:

\begin{equation*}
d_{\texttt{Euclidean}} = \sqrt{\sum_i{(x_{0,i} - x_{1,i})^2}}
\end{equation*}

Implement the Euclidean distance as a Python function:
- `euclidean_distance`: calculates the Euclidean distance between two n-dimensional samples

    *Inputs:*<br>
    `x1`: An n-dimensional sample. This either a list of length $n$, or a `numpy.array` of dimensions `(n,)`<br>
    `x2`: An n-dimensional sample. This either a list of length $n$, or a `numpy.array` of dimensions `(n,)`<br>

    *Returns*<br>
    The Euclidean distance between `x1` and `x2`.<br>
    
    *Example:*

    ```python
    x1 = [7, -1]
    x2 = [4, 3]
    print(euclidean_distance(x1, x2)
    ```
    *Example output:*<br>
    `5.0`

    

#### SOLUTION CELL:

In [ ]:
def euclidean_distance(x1, x2):
                
    ################################
    ####### YOUR CODE HERE #########
    ################################


#### TESTING CELL:

In [ ]:
a = [7, -1]
b = [4, 3]
print("Euclidean distance between {} ({}D) and {} ({}D) is {}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = [1, 2]
b = [3, 4]
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = [4.2, 3.6, 1.0]
b = [5.7, 2.8, -1.5]
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = np.random.rand(20)
b = np.random.rand(20)
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))

### Task 2.2 - kNN classifier
#### 20% marks

K nearest neighbours classifier will predict the class of a sample based on the class label of its $k$ nearest neigbours, according to a given distance metric. Implement `kNN` classifier which works on samples with numerical features, and relies on the Euclidean distance, as a Python class implementing the following member functions:

- `__init__(self, k = 5, distance = euclidean_distance)`: class constructor

    *Inputs:*<br>
    `k` : The number of neigbours to be considered by the classification model <br>
    `distance`: Tunction which is used to calculate the distance between samples (you will use the Euclidean distance implemented in the previous task)<br><br>
    
- `fit(self, X, y)`: member function used to train the classifier (fit the model to the data)

    *Inputs:*<br>
    `X`: The dataset samples. This is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, n)`<br>
    `y`: The dataset labels. This is either a list of length $M$, or a `numpy.array` of dimensions `(M,)`<br><br>
    
    *Returns:*<br>
    The trained classifier model.<br><br>
    
- `predict(self, X)`: member function used to predict the classes of samples `X`

    *Inputs:*<br>
    `X`: The samples for which to predict a class. If this is a single sample, then `X` is either a list of length $n$, or a `numpy.array` of dimensions `(n,)`. If this is a set of samples, then `X` is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, N)`<br>
    
    *Returns:*<br>
    Predicted class for the samples of X. If `X` was a single sample, then the output is a single class label. If `X` was a set of samples, then the output is a `numpy.array` of dimensions `(n,`)<br>
    
    *Note:*<br>
    The provided code outline already handles the case where `X` is a set of samples (by running on each sample separately). You need write the main logic of this function, i.e. calculating the prediction for a single sample.<br><br>

#### SOLUTION CELL:

In [ ]:
class knn:
    def __init__(self, k = 5, distance = euclidean_distance):
        
        ################################
        ####### YOUR CODE HERE #########
        ################################
       
    def fit(self, X, y):
        
        ################################
        ####### YOUR CODE HERE #########
        ################################

        return self # this should probably be the last line in your fit function
            
    def predict(self, X):
        
        # the fololwing block of code handles lists/arrays containing multiple samples
        X = np.array(X)
        if len(X.shape) >= 2:
            return np.array([self.predict(x) for x in X])
        
        # in the following section, you can assume that X is a single n-dimensional sample:
        ################################
        ####### YOUR CODE HERE #########
        ################################

        return 0 # you may replace this line
        

#### TESTING CELL:

In [ ]:
# Initialise the classifier with k=3 (using 9 neigbours)
classifier = knn(k=9)
# Fit the classifier to the training set
classifier.fit(X_nd_train, y_nd_train)

# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_nd_test[3:10], classifier.predict(X_nd_test[3:10]), y_nd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

## Decision Trees

For the second part of the assignment, you are expected to implement a decision tree classifier. A decision tree models the data as a series of decisions based on the values of different features of the sample. The predictions are made in an interpretable fashion, where the decision is made by considering different sample features, and their values, until the new sample is grouped with similar samples presented during training.

### Task 3.1 - Entropy and Information Gain
#### 10% marks

Important concepts in decision trees are **entropy** and **information gain**. **Entropy** is a measure of the variance in the data, of how "unpredictable" the dataset is:

- If a dataset consists only of samples with the same label (e.g. $\{\blacksquare,\blacksquare,\blacksquare,\blacksquare,\blacksquare,\blacksquare\}$), the entropy will be zero (the data is predictable).
- If a dataset consists of an equal amount of samples from two classes, (e.g. $\{\blacksquare,\triangle,\triangle,\blacksquare,\triangle,\blacksquare\}$), the entropy will be one (the data is unpredictable).
- If the class distribution in the dataset lies somewhere between these two extremes, the entropy will be between one and zero.
- Entropy also increases as the number of different classes increases.

Given $c$ classes, and their associated probabilities (frequencies) in the datasets, $p_i \forall i \in 0..c$, the entropy is calculated as:

\begin{equation*}
E = -\sum_i p_i log_2(p_i).
\end{equation*}

Implement a function `entropy` which calculates the entropy of a set of class labels from the list of their associated probabilities:

- `entropy(p_per_class)`: calculates the entropy of a set of class probabilities

    *Inputs:*<br>
    `p_per_class`: A list of $c$ probabilities, one for each class in the dataset. This either a list of length $c$, or a `numpy.array` of dimensions `(c,)`<br>

    *Returns:*<br>
    The entropy of a set with class probabilities given in `p_per_class`.<br>
    
    *Note:*<br>
    You do not need to check if the list of probabilities given is correct; i.e. you may assume that all the probabilities in the list are strictly greater than zero, and that the sum of all the class probabilities equals one.
    
    *Example:*

    ```python
    probabilities = [0.5, 0.5]
    print(entropy(probabilities))
    ```
    *Example output:*<br>
    `1.0`

#### SOLUTION CELL:

In [ ]:
def entropy(p_per_class):
    ################################
    ####### YOUR CODE HERE #########
    ################################

    return 0 # you may replace this line
    

#### TESTING CELL:

In [ ]:
print("Entropy of a set with only one class: {}".format(entropy([1.0])))
print("Entropy of a set with two equally probable classes: {}".format(entropy([0.5, 0.5])))
print("Entropy of a set with four equally probable clasess: {}".format(entropy([0.25, 0.25, 0.25, 0.25])))
print("Entropy of a set with four classes with different probabilities: {:.3f}".format(
    entropy([0.2, 0.3, 0.4, 0.1])))
print("Entropy of the numerical (batteries) dataset: {:.3f}".format(
    entropy(list(class_probabilities(y_nd).values()))))
print("Entropy of the categorical (forest cover) dataset: {:.3f}".format(
    entropy(list(class_probabilities(y_cd).values()))))

The **information gain** describes how much variance was lost by dividing a set into parts, i.e. how much more "predictable" the parts are from the whole.

For example, if we have a dataset $d=\{\blacksquare,\triangle,\triangle,\blacksquare,\triangle,\blacksquare,\triangle,\blacksquare\}$ (which is "maximally unpredictable", so we can calculate $E(d) = 1$), we can:
- split it into $d_1=\{\blacksquare,\blacksquare,\blacksquare,\blacksquare\}$ and $d_2=\{\triangle, \triangle, \triangle, \triangle\}$.

    Both of $d_1$ and $d_2$ are "very predictable" ($E(d_1) = E(d_2) = 0$), both subsets consist of one class only). The resulting information gain is large (we go from unpredictable to predictable, so we gain information):
    
    $I = E(d) - (0.5E(d_1) + 0.5E(d_2) = 1 - (0+0) =1$
    
- split it into $d_1=\{\blacksquare,\triangle,\triangle,\blacksquare\}$ and $d_2=\{\triangle,\blacksquare,\triangle,\blacksquare\}$.

    Both of $d_1$ and $d_2$ are "very unpredictable" ($E(d_1) = E(d_2) = 1$), both subsets have the same amount of samples from each of the two classes). The resulting information gain is small (we start from an unpredictable dataset, and end with two unpredictable subsets, so we do not gain information):

    $I = E(d) - (0.5E(d_1) + 0.5E(d_2) = 1 - (0.5\times1+0.5\times1) =0$
    
<br>
In general, if we have a dataset $d$ containing $|d|$ samples, and we split it into $s$ subsets $d_i \forall i=1..s$, each of size $|d_i|$, the information gain of this split is calculated as:

\begin{equation*}
    I = E(d) - \sum_{i=0}^s \frac{|d_i|}{|d|}E(d_i)
\end{equation*}

### Task 3.2 - Decision Tree classifier
#### 30% marks

In each decision node of a decision tree, this classifier splits the dataset into two or more parts according to a value of a certain feature in the dataset.

Given a set of M samples $\mathbf{x} = \{\mathbf{x}_i\} \forall i=1..M$ with n features each $\mathbf{x}_i = \{x_{1,i}, \dots, x_{n,i}\}$, we could decide to split the dataset at the $j^{\texttt{th}}$ (numerical) feature at value $v$. This would mean splitting the dataset into:
- the subset $\mathbf{x_{<}} = \{\mathbf{x}_i | x_{j,i} < v\}$, which contains all the samples with the $j^{\texttt{th}}$ smaller than $v$ and
- the subset $\mathbf{x_{\geq}} = \{\mathbf{x}_i | x_{j,i} \geq v\}$, which contains all the samples with the $j^{\texttt{th}}$ greater or equal than $v$.

Similarly, we could decide to split a dataset at the $j^{\texttt{th}}$ feature which is categorical. If the $j^{\texttt{th}}$ feature of a sample $\mathbf{x}_i$ can hold values $\{\texttt{child}, \texttt{teen}, \texttt{adult}\}$, the resulting subsets would be:
- the subset $\mathbf{x_{\texttt{child}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{child}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{child}$, 
- the subset $\mathbf{x_{\texttt{teen}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{teen}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{teen}$ and
- the subset $\mathbf{x_{\texttt{adult}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{adult}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{adult}$.

In each node of the decision tree, the dataset is split according to the attribute $j$ which results in **the highest information gain** after the split. For categorical features, there is only one possible split (each category becomes a subset). For numerical features, it is neccessary to check all the possible values of split. For example, if the $j^{\texttt{th}}$ in the dataset has appeared with the values of $\{1, 1.5, 2.7, 3.2, 5\}$, you need to calculate the information gain for splitting the dataset at $v=1.5$ (into $\mathbf{x_{< 1.5}}$ and $\mathbf{x_{\geq 1.5}}$), $v=2.7$, $v=3.2$, $v=5$.

The splitting continues until each node contains only samples of a single class, or a stopping criterion is reached. If a node contains training samples of more than one class, it returns the label of the majority (most probable) class as its decision.

Implement a `DecisionTree` classifier which works on samples with both numerical and categorical features as a Python class implementing the following member functions:

- `__init__(self, max_depth = -1, min_samples_per_node = 1)`: class constructor

    *Inputs:*<br>
    `max_dept` : The maximal depth of the tree (`max_depth == 0` corresponds to a decision tree with only the root node. `max_depth == -1` means no limit to the depth of the tree) <br>
    `min_samples_per_node`: The minimal number of samples contained in a terminal (decision) node. If a split would cause one of the subsets to have less than `min_samples_per_node` samples, the split is not performed. <br><br>
    
- `fit(self, X, y)`: member function used to train the classifier (fit the model to the data)

    *Inputs:*<br>
    `X`: The dataset samples. This is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, n)`<br>
    `y`: The dataset labels. This is either a list of length $M$, or a `numpy.array` of dimensions `(M,)`<br>
    
    *Returns:*<br>
    The trained classifier model.<br><br>
    
- `predict(self, X)`: member function used to predict the classes of samples `X`

    *Inputs:*<br>
    `X`: The samples for which to predict a class. If this is a single sample, then `X` is either a list of length $n$, or a `numpy.array` of dimensions `(n,)`. If this is a set of samples, then `X` is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, N)`<br>
    
    *Returns:*<br>
    Predicted class for the samples of X. If `X` was a single sample, then the output is a single class label. If `X` was a set of samples, then the output is a `numpy.array` of dimensions `(n,`)<br>
    
    *Note:*<br>
    The provided code outline already handles the case where `X` is a set of samples (by running on each sample separately). You need write the main logic of this function, i.e. calculating the prediction for a single sample.<br><br>
    
*Note:* Your implementation will be tested in two parts. Firstly, it will be tested only on a dataset containing numerical values. Secondly, it will also be tested on a dataset containing a mix of categorical and numerical values.

#### SOLUTION CELL:

In [ ]:
class DecisionTree:
    def __init__(self, max_depth = -1, min_samples_per_node = 1):
        ################################
        ####### YOUR CODE HERE #########
        ################################
           
    def fit(self, X, y):
        ################################
        ####### YOUR CODE HERE #########
        ################################
        
        return self # you may replace this line
               
    def predict(self, X):

        # the fololwing block of code handles lists/arrays containing multiple samples
        X = np.array(X)
        if len(X.shape) >= 2:
            return np.array([self.predict(x) for x in X])
        
        # in the following section, you can assume that X is a single n-dimensional sample:
        ################################
        ####### YOUR CODE HERE #########
        ################################
        

#### TESTING CELLS:

The following cell will test if the classifier implemented model can fit to the [numerical dataset](#Numerical-data), and predict solutions for some of the testing samples (this is not model evaluation yet):

In [ ]:
# Initialise the classifier, requiring at least 3 samples in each node
classifier = DecisionTree(min_samples_per_node = 3)
# Fit the classifier to the training data
classifier.fit(X_nd_train, y_nd_train)
# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_nd_test[3:10], classifier.predict(X_nd_test[3:10]), y_nd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

The following cell will check if the dataset is able to fit the model to the [dataset with mixed numerical and categorical values](#Categorical-data), and predict solutions for some of the testing samples (this is not model evaluation yet):

_**Warning:**_ Executing this cell may take a bit of time.

In [ ]:
# Initialise the classifier, requiring the decision tree to have no more than 5 levels
classifier = DecisionTree(max_depth = 4)
# Fit the classifier to the training data
classifier.fit(X_cd_train, y_cd_train)

# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_cd_test[3:10], classifier.predict(X_cd_test[3:10]), y_cd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

## Model evaluation and analysis

Executing all the cells from the previous section correctly only means that your implementation of [kNN classifier](#knn-Classifier) and [Decision Tree classifier](#Decision-Tree-classifier) runs on the provided data. This has still not provided any insight on how well those models represent the data.

In this section, you are instead required to calculate some evaluation metrics, and evaluate the produced models on the whole of the held-out data (test set) defined in the [Dataset](#Dataset) section.

### Task 4.1 - Evaluation
#### 10% marks

You are required to write a function that will calculate different evaluation metrics based on the ground truth labels and the predicted labels. Specifically, write a function:
- `evaluate(y_true, y_pred)`: evaluate classification results

    *Inputs:*<br>
    `y_true` : Ground truth labels for $n$ samples. This is either a list of $n$ elements, or a `numpy.array` of dimensions `(n,)`
    `y_pred` : Predicted labels for $n$ samples. This is either a list of $n$ elements, or a `numpy.array` of dimensions `(n,)`
    
    *Returns:*<br>
    A touple `(a, p, r, k)` consisting of the following values:
    - `a` : overall accuracy of the predictions
    - `p` : a dictionary containing a precision score for each class present in `y_true`
    - `r` : a dictionary containing a recall score for each class present in `y_true`
    - `k` : Cohen's Kappa metric

#### SOLUTION CELL:

In [ ]:
from sklearn import metrics

def evaluate(y_true, y_pred):
    
    ################################
    ####### YOUR CODE HERE #########
    ################################
    
    return 0,{},{},0 # you may replace this line
    

#### TESTING CELL:

The following cell evaluates your implementation of [kNN classifier](#Task-2.2---kNN-classifier) and [Decision Tree classifier](#Task-3.2---Decision-Tree-classifier) on the numerical dataset defined in the [Dataset](#Dataset) section:

In [ ]:
classifier_knn = knn(k=5)
classifier_knn.fit(X_nd_train, y_nd_train)

classifier_dt = DecisionTree(min_samples_per_node = 3)
classifier_dt.fit(X_nd_train, y_nd_train)

y_knn = classifier_knn.predict(X_nd_test)
y_dt = classifier_dt.predict(X_nd_test)

a_knn, p_knn, r_knn, k_knn = evaluate(y_nd_test, y_knn)
a_dt, p_dt, r_dt, k_dt = evaluate(y_nd_test, y_dt)

print("Evaluating numerical dataset, using kNN:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_knn, k_knn))
for key in p_knn.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_knn[key], r_knn[key]))

print("Evaluating numerical dataset, using DecisionTree:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_dt, k_dt))
for key in p_dt.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_dt[key], r_dt[key]))

The following cell evaluates your implementation of [Decision Tree classifier](#Decision-Tree-classifier) on the [dataset containing a mixture of numerical and categorical data](#Categorical-data):

_**Warning:**_ Executing this cell may take a bit of time.

In [ ]:
classifier_categorical = DecisionTree(max_depth = 4)
classifier_categorical.fit(X_cd_train, y_cd_train)

y_cd_pred = classifier_categorical.predict(X_cd_test)
a_cat, p_cat, r_cat, k_cat = evaluate(y_cd_test, y_cd_pred)

print("Evaluating categorical dataset, using DecisionTree:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_cat, k_cat))
for key in p_cat.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_cat[key], r_cat[key]))


### Task 4.2 - Analysis
#### 10% marks

Answer the following questions in the space provided below (marked as **YOUR ANSWERS**):
1. Explain the difference between the different performance metrics calculated for [Task 4.1](#Task-4.1---Evaluation). Which additional metrics could you propose to evaluate the performance of the defined models? Which of the calculated metrics would you chose to report, and why, for the analysis of the two [datasets](#Datasets) used in this assessment?

2. In case you were handling a _very large_ amount of data, which of the models do you expect to be larger (i.e. take up more memory)? Which of the models do you expect to train faster? Which of the models do you expect to reach a decision faster? Explain and justify your answers.

3. When handling numerical data, what is the role of normalisation? The numerical data used in this assessment was not normalised. If you were to normalise the data before training these classification models, would either of them change how they represent the data (and potentially return different results)?

4. For evaluating the performance of the classifiers implemented for this assignment, $80\%$ of the data was used for training the models, and $20\%$ for testing. What are the shortcomings of this evaluation strategy, and can you propose and describe a better one?

_Note:_ You may use the cell marked as **EXPERIMENTAL CELL** to write any code that could help you answer the above questions, or in general, to experiment with the code produced for this assessment.

#### EXPERIMENTAL CELL:

In [ ]:
############ You may use this cell to write any code which may help you answers the questions above ##############
####### Your implementations from this cell will not be assessed, feel free to use it for experimentation ########

#### YOUR ANSWERS:

1. <font color='red'>Answer to question 1</font> 
2. <font color='red'>Answer to question 2</font> 
3. <font color='red'>Answer to question 3</font> 
4. <font color='red'>Answer to question 4</font> 